In [2]:
from flask import jsonify
import requests
import datetime
import pandas as pd
from model.LSTMModel import LSTMModel
from model.RNNModel import RNNModel
from model.XgboostModel import XGBoostModel

def fetch_binance_data(symbol="BTCUSDT", interval="5m", limit=200):
    url = f"https://api.binance.us/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "limit": limit
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        # get last row of data
        data = response.json()
        df = pd.DataFrame(data, columns=['t', 'o', 'h', 'l', 'c', 'v', 'T', 'q', 'n', 'V', 'Q', 'B'])
        df = df[['t', 'o', 'h', 'l', 'c']]
        df['t'] = pd.to_datetime(df['t'], unit='ms')
        print(df)
        
        # Load model
        model = XGBoostModel(symbol)
        model.load('./trained/BTCUSDT-XGBoost_Close')

        if model:
            # Ensure column names match the expected names used in the model prediction
            df.rename(columns={'c': 'Close'}, inplace=True)
            df.rename(columns={'t': 'Datetime'}, inplace=True)
            df.rename(columns={'o': 'Open'}, inplace=True)
            df.rename(columns={'h': 'High'}, inplace=True)
            df.rename(columns={'l': 'Low'}, inplace=True)
            
            predictions = model.predict(df[['Datetime','Open','High', 'Low','Close']])
            print(f' prediction: {predictions}')
            # Format predictions as required ([[t, o, h, l, c], [t, o, h, l, c], ...])
            prediction_list = []
            for prediction in predictions:
                prediction_list.append([ prediction['Open'], prediction['High'], prediction['Low'], prediction['Close']])

            print(f' Prediction list: {prediction_list}')
           
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Example usage
data = fetch_binance_data()

                      t               o               h               l  \
0   2024-07-15 14:45:00  62931.61000000  62994.04000000  62906.30000000   
1   2024-07-15 14:50:00  62989.75000000  63090.50000000  62989.75000000   
2   2024-07-15 14:55:00  63015.03000000  63205.41000000  63015.03000000   
3   2024-07-15 15:00:00  63168.69000000  63257.93000000  63168.69000000   
4   2024-07-15 15:05:00  63207.06000000  63215.30000000  62892.40000000   
..                  ...             ...             ...             ...   
195 2024-07-16 07:00:00  62933.16000000  63103.80000000  62814.44000000   
196 2024-07-16 07:05:00  63132.08000000  63227.96000000  63110.59000000   
197 2024-07-16 07:10:00  63241.12000000  63295.53000000  63174.70000000   
198 2024-07-16 07:15:00  63183.90000000  63229.20000000  63129.70000000   
199 2024-07-16 07:20:00  63037.78000000  63037.78000000  63037.78000000   

                  c  
0    62978.96000000  
1    63057.31000000  
2    63161.03000000  
3    63257.

KeyError: "['ROC'] not in index"